In [52]:
import sys
import os
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append(os.path.abspath('..'))
from credmodex.credlab import CredLab

from sklearn.linear_model import LogisticRegression
from optbinning import OptimalBinning
from credmodex.discriminancy.discrete import Correlation
from credmodex.utils.design import plotly_main_layout
from credmodex.discriminancy.discrete import PSI_Discriminant

In [62]:
df = {
    'Grade': [0]*(95+309) + [1]*(187+224) + [2]*(549+299) + [3]*(1409+495) + [4]*(3743+690) + [5]*(4390+424) + [6]*(2008+94) + [7]*(593+8),
    'Target': [0]*95+[1]*309 + [0]*187+[1]*224 + [0]*549+[1]*299 + [0]*1409+[1]*495 + [0]*3743+[1]*690 + [0]*4390+[1]*424 + [0]*2008+[1]*94 + [0]*593+[1]*8
}
df = pd.DataFrame(df)

In [149]:
df = pd.read_csv(r'df.csv')
df['DATA'] = df['DATA'].astype('datetime64[ns]')

In [55]:
features = ['idade','score_scr','total_consultas','total_spc','cidade_loja_cliente']

project = CredLab(
    df, target='over', features=features, time_column='DATA',
    test_size=0.2, split_type='random'
)

In [56]:
# def treatment_func(df):
#     df = project.df.copy()
#     df = df.dropna()
#     return df

# project.add_model(
#     model=LogisticRegression(max_iter=1000, solver='saga'),
#     treatment=treatment_func,
# )

In [57]:
# project.LogisticRegression_1.add_rating(
#     model=OptimalBinning(dtype="numerical", solver="cp", max_n_bins=7)
# )

In [ ]:
PSI_Discriminant(df, 'over', features).plot('score_scr',)

In [ ]:
from optbinning import BinningProcess, OptimalBinning
import pandas as pd
import numpy as np

def build_gini_table_optbinning(df, score_col, target_col):
    if pd.api.types.is_datetime64_any_dtype(self.df[col]):
        return None

    if (is_continuous) or (self.df[col].dtype == 'float'):
        # Create bins based on training data
        binning = OptimalBinning(name=col, dtype="numerical", max_n_bins=max_n_bins)
        binning.fit(self.train[col].dropna(), y=self.train[self.train[col].notna()][self.target])

        # Apply binning to train and test sets
        train_binned = binning.transform(self.train[col], metric="bins")
        test_binned = binning.transform(self.test[col], metric="bins")

        # Convert to categorical for grouping
        train = pd.Series(train_binned).value_counts(normalize=True).sort_index().rename("Reference")
        test = pd.Series(test_binned).value_counts(normalize=True).sort_index().rename("Posterior")
    else:
        # Use categorical value counts
        train = self.train[col].value_counts(normalize=True).rename('Reference')
        test = self.test[col].value_counts(normalize=True).rename('Posterior')



# Assuming your DataFrame is called df and has 'score' and 'target' columns
gini_table, gini_value = build_gini_table_optbinning(df, score_col='score_scr', target_col='over')

NameError: name 'self' is not defined

In [ ]:
gini_table

,Grade,Good,Bad,Odds,CumGood%,CumBad%,FormulaGood,FormulaBad,Product
0,0,27300,15258,1.8,13.5%,13.8%,13.5,13.8,186.7
1,1,32387,24986,1.3,29.5%,36.5%,29.5,36.5,1076.8
2,2,42658,28525,1.5,50.6%,62.3%,50.6,62.3,3155.1
3,3,10404,6296,1.7,55.8%,68.0%,55.8,68.0,3794.0
4,4,19108,9421,2.0,65.2%,76.6%,65.2,76.6,4993.9
5,5,13789,6339,2.2,72.0%,82.3%,72.0,82.3,5930.0
6,6,11375,4867,2.3,77.7%,86.7%,77.7,86.7,6735.7
7,7,12174,4724,2.6,83.7%,91.0%,83.7,91.0,7616.3
8,8,20035,6751,3.0,93.6%,97.1%,93.6,97.1,9090.8
9,9,12958,3163,4.1,100.0%,100.0%,100.0,100.0,10000.0
